In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account
key_path = "./creds/service_acc_auth.json"
credentials = service_account.Credentials.from_service_account_file(key_path, scopes = ["https://www.googleapis.com/auth/cloud-platform"])
bigquery_client = bigquery.Client(credentials=credentials, project=credentials.project_id)

In [2]:
# query new data <= 1/1
union_weeks = """
SELECT * , 
FROM `scanning-database.scanning_data_backup.W42`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W43`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W44`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W45`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W46`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W47`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W48`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W49`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W50`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W51`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W52`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W53`
"""

In [3]:
# Overwriting old PowerBI source data
job_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10,
destination = 'scanning-database.PowerBI.PowerBI',
write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
) # set query limit to 10G
query_job = bigquery_client.query(union_weeks, job_config)  # API request
rows = query_job.result()  # Waits for query to finish
data_in_use = rows.to_dataframe()

In [4]:
data_in_use.sort_values(by=['Week']).tail()

,Start_Time,Finish_Time,Activity,Seq_Code,Recipe_Name,Break_Reasons,Missing_Ingredients,Kitting_Line,Assembly_Batch,Event_Shift,Team_Leader,Pickers_Count,Time_Consumption,Week
32810,2020-12-24 15:35:44+00:00,2020-12-24 15:36:04+00:00,Production,0001 of 0001,Q4,None,None,KL9,Sunday Assembly,Thursday Afternoon,iXUsr_Melia(8310),1,0.33,2020-53
60904,2020-12-24 10:09:52+00:00,2020-12-24 10:15:17+00:00,Production,0019 of 0022,D2,None,None,KL1,Saturday Assembly,Thursday Morning,iXUsr_Siregar(6092),3,5.42,2020-53
32808,2020-12-24 18:59:34+00:00,2020-12-24 18:59:47+00:00,Production,0001 of 0001,U2,None,None,KL15,Sunday Assembly,Thursday Afternoon,iXUsr_Melia(8310),1,0.22,2020-53
10366,2020-12-27 09:10:22+00:00,2020-12-27 09:13:41+00:00,Production,0040 of 0062,C4,None,None,KL6,Monday Assembly,Sunday Morning,iXUsr_Yin(9596),3,3.32,2020-53
53158,2020-12-24 07:26:27+00:00,2020-12-24 07:29:43+00:00,Production,0002 of 0019,H2,None,None,KL11,Saturday Assembly,Thursday Morning,iXUsr_Tafai(10143),3,3.27,2020-53


In [5]:
data_in_use.shape

(79454, 14)